# ML Project

## Setup

## Install Packages

In [1]:
#%%capture
!python3 -m pip install -U pandas numpy matplotlib tensorflow tqdm bs4 IP2Location chardet scikit-learn

Requirement already up-to-date: pandas in ./env/lib/python3.8/site-packages (2.0.1)
Requirement already up-to-date: numpy in ./env/lib/python3.8/site-packages (1.24.3)
Requirement already up-to-date: matplotlib in ./env/lib/python3.8/site-packages (3.7.1)
Requirement already up-to-date: tensorflow in ./env/lib/python3.8/site-packages (2.12.0)
Requirement already up-to-date: tqdm in ./env/lib/python3.8/site-packages (4.65.0)
Requirement already up-to-date: bs4 in ./env/lib/python3.8/site-packages (0.0.1)
Requirement already up-to-date: IP2Location in ./env/lib/python3.8/site-packages (8.10.0)
Requirement already up-to-date: chardet in ./env/lib/python3.8/site-packages (5.1.0)
Requirement already up-to-date: scikit-learn in ./env/lib/python3.8/site-packages (1.2.2)


### Import Modules

In [2]:
import requests
import urllib3
from urllib.parse import urlparse, urljoin
import socket

from bs4 import BeautifulSoup
import chardet
import re
import json
import ast

import math
import statistics
from matplotlib import pyplot

import os
import IP2Location

import pandas as pd
import numpy as np

import random
import itertools

from tqdm import tqdm

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
print(tf.__version__)

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

import warnings
warnings.filterwarnings(action="ignore", module="urllib3")

2023-04-25 13:18:41.375099: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 13:18:41.456456: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 13:18:41.457934: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 13:18:42.662962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.12.0


## Download More Data / Feature Engineering

### Set Constants

In [3]:
DOWNLOAD_CHUNK_SIZE = 0
TIMEOUT = (10, 10)
feature_names = ["words", "aux", "city", "region", "country", "redirects", "latitude", "longitude"]

### Define Helper Functions

In [4]:
ip2location_database = IP2Location.IP2Location()
ip2location_database.open(os.path.join("location_data", "IP2LOCATION-LITE-DB11.BIN"))
def get_location(ip_addr=None, hostname=None):
    if(ip_addr is None and hostname is not None):
        try:
            ip_addr = socket.gethostbyname(hostname)
        except socket.gaierror:
            print("Skipped Location Download (Hostname Resolution Error for '"+hostname+"')")
            return None
    location_data = ip2location_database.get_all(ip_addr)
    if(location_data.country_short == "-"):
        if(not ip_addr in location_database):
            location_data = requests.get("https://ipinfo.io/"+ip_addr+"/json").json()
            if("error" in location_data):
                raise Exception("Failed because error with download (probably api quota exceded)")
            location_database[ip_addr] = location_data
            location_database[ip_addr]["country_short"] = location_data["country"]
            location_database[ip_addr]["latitude"], location_database[ip_addr]["longitude"] = tuple(location_data["loc"].split(","))
    else:
        return ast.literal_eval(str(location_data))
    return location_database[ip_addr]

In [5]:
def get_absolute_url_from_relative(original_url, new_url):
    if(new_url.startswith("javascript")):
        return None
    if(new_url.startswith("http")):
        return new_url
    url_with_scheme = "http://"+original_url if not original_url.startswith("http") else original_url
    parsed_url = urlparse(url_with_scheme)
    url_scheme = parsed_url.scheme
    url_host = parsed_url.netloc
    
    return urljoin(url_scheme+"://"+url_host, new_url)

In [6]:
def download_aux_data(content, original_url):
    aux = []
    soup = BeautifulSoup(content, 'html.parser')
    for a in soup.find_all('a'):
        href = a.get('href')
        if(href is not None):
            url = get_absolute_url_from_relative(original_url, href)
            if(url is not None):
                hostname = urlparse(url).netloc
                json = get_location(hostname=hostname)
                if(json is None):
                    continue
                try:
                    columns = [
                        "hostname_"+hostname,
                        "ip_addr_"+json["ip"],
                        "latitude_"+str(int(float(json["latitude"]))),
                        "longitude_"+str(int(float(json["longitude"]))),
                        "city_"+json["city"],
                        "region_"+json["region"],
                        "country_"+json["country_short"],
                    ]
                except KeyError:
                    if("bogon" in json and json["bogon"]):
                        print("Skipped Aux Data Download (Bogon IP)")
                    else:
                        print("Problem with JSON: ", json)
                    columns = []
                
                columns = [ re.compile('[\W_]+').sub('_', column) for column in columns ]
                
                aux = aux + columns
                
    return " ".join(aux)

In [7]:
def download_one(original_url, timeout=TIMEOUT):
    row = {}
    row["attempted_download"] = True

    url_with_scheme = "http://"+original_url if not original_url.startswith("http") else original_url
    parsed_url = urlparse(url_with_scheme)
    hostname = parsed_url.netloc
    
    try:
        row["hostname"] = hostname
        
        ip_addr = socket.gethostbyname(hostname)
        row["ip_addr"] = ip_addr

        r = requests.get(url_with_scheme, verify=False, timeout=timeout)

        row["status_code"] = r.status_code
        encoding = chardet.detect(r.content)['encoding']
        #if(encoding == None):
        #    print("Skipped content download (Decoding Error)")
        try:
            content = r.content.decode(encoding)
            row["content"] = content
        except:
            try:
                content = r.content.decode("utf")
            except:
                content = None
                print("Skipped Content Download (Decoding Error)")
            
        if(content is not None):
            row["aux"] = download_aux_data(content, original_url)
            row["words"] = " ".join(re.compile('[\W_]+').sub(' ', BeautifulSoup(content, 'html.parser').get_text()).split(" "))
        
        redirects = 0
        for r_history in r.history:
            if(r_history.status_code == 301):
                redirects = redirects + 1
        row["redirects"] = redirects

        json = get_location(ip_addr=ip_addr)
        if(json is None):
            return row
        try:
            row["latitude"] = json["latitude"]
            row["longitude"] = json["longitude"]
            row["city"] = json["city"]
            row["region"] = json["region"]
            row["country"] = json["country_short"]

        except KeyError:
            if("bogon" in json and json["bogon"]):
                print("Skipped Location Download (Bogon IP)")
            else:
                print("Problem with JSON: ", json)
        
    except socket.gaierror:
        print("Skipped (Hostname Resolution Error for '"+hostname+"')")
                      
    except socket.error:
        print("Skipped (Content Download Error for '"+original_url+"')")
        
    except UnicodeError:
        print("Skipped (Unicode Error for '"+original_url+"')")
    
    return row

In [8]:
def download_chunk(chunk_size=DOWNLOAD_CHUNK_SIZE, timeout=TIMEOUT):
    global raw_data
    if(len(raw_data[raw_data["attempted_download"] == True].index) != 0):
          start = raw_data[raw_data["attempted_download"] == True].index[-1]+1
    end = start + chunk_size
    end = end if len(raw_data["url"]) > end else len(raw_data["url"])
    
    print("Downloading %d more rows ([%d:%d])" % (chunk_size, start, end))
    for row_index in tqdm(range(start, end)):
        row = download_one(raw_data.loc[row_index, "url"], timeout=timeout)
        if(len(list(row)) > 0):
            raw_data.loc[row_index, list(row)] = row.values()

In [9]:
def load_data():
    global raw_data, location_database
    
    raw_data = pd.read_csv("./raw_data.csv")
    raw_data["attempted_download"] = False
    
    try:
        with open("./location_data.json", 'r') as file:
            location_database = json.load(file)
        file.close()
    except FileNotFoundError:
        location_database = {}

    try:
        raw_data = pd.read_csv("./data.csv", index_col=0, low_memory=False)
        if(DOWNLOAD_CHUNK_SIZE > 0):
            download_chunk()
            raw_data.to_csv("./data.csv")
            with open("./location_data.json", 'w') as file:
                json.dump(location_database, file)
            file.close()

    except FileNotFoundError:
        download_chunk()
        raw_data.to_csv("./data.csv")
        with open("./location_data.json", 'w') as file:
            json.dump(location_database, file)
        file.close()

    data = raw_data.copy()

    data = data.drop("attempted_download", axis=1)
    data = data[data["status_code"] == 200]
    data = data.drop("status_code", axis=1)
    data = data[feature_names + ["type", "url"]].dropna()
    data = data.reset_index(drop=True)
    data["redirects"] = data["redirects"].astype(str)
    data["latitude"] = data["latitude"].astype(str)
    data["longitude"] = data["longitude"].astype(str)
    return data

### Load/Download Data and display data

In [10]:
data = load_data()
data

,words,aux,city,region,country,redirects,latitude,longitude,type,url
0,larcadelcarnevale com Buy this domain larcade...,hostname_secure_voodoo_com ip_addr_192_64_146_...,Munich,Bavaria,DE,0.0,48.1374,11.5755,defacement,http://larcadelcarnevale.com/catalogo/palloncini
1,Sorteo Notebook Diciembre 2012JavaScript isn t...,hostname_accounts_google_com ip_addr_142_251_1...,Washington,"Washington, D.C.",US,2.0,38.8951,-77.0364,phishing,https://docs.google.com/spreadsheet/viewform?f...
2,Shopper s Heaven 인터파크 홈 리빙 세탁 청소용품 리빙 최신 등록순 ...,hostname_interpark_com ip_addr_211_233_74_23 l...,Seoul,Seoul,KR,0.0,37.566,126.9784,benign,http://interpark.com/displaycorner/FreeMarket....
3,Links KontaktAnfahrtDatenschutzImpressum Home...,hostname_www_pn_wuppertal_de ip_addr_217_160_0...,Karlsruhe,Baden-Wurttemberg,DE,1.0,49.0094,8.4044,defacement,http://www.pn-wuppertal.de/links/2-linkseite/5...
4,AfterMarket pl domena parafiapiaski pl Domena...,hostname_www_aftermarket_pl ip_addr_185_253_21...,Warsaw,Mazovia,PL,0.0,52.2298,21.0118,defacement,http://www.parafiapiaski.pl/index.php?option=c...
...,...,...,...,...,...,...,...,...,...,...
784,WebopRo com is for sale HugeDomains Search 1 ...,hostname_www_HugeDomains_com ip_addr_104_26_6_...,Ashburn,Virginia,US,0.0,39.039474,-77.491806,defacement,http://www.webopro.com/index.php/sits/pwws/119...
785,Teaspoon A Man Who Wasn t There An Alt Series...,hostname_whofic_com ip_addr_179_61_137_3 latit...,Victoria,Texas,US,1.0,28.805269,-97.003601,benign,whofic.com/series.php?seriesid=1745
786,Evri The New Hermes Cheap Parcel Delivery Cou...,hostname_evri_com ip_addr_45_60_6_42 latitude_...,Seattle,Washington,US,2.0,47.603909,-122.329842,benign,evri.com/organization/kukc-lp-0x1364cd
787,SelamSoft Amharic Dictionary A Your search wa...,hostname_amharicdictionary_com ip_addr_50_21_1...,Philadelphia,Pennsylvania,US,0.0,39.96244,-75.199928,defacement,http://amharicdictionary.com/default.aspx


## Model Building

### Define Hyperparameter Constants

In [11]:
max_features = 10000
sequence_length = 1000
epochs = 25
folds = 5
batch_size = 1
test_size = 0.33
validation_size = 0.20 # ratio after test has been taken out
seed = 123

### Ensure Reproducibility (important for feature subsets comparing)

In [12]:
def reset_random_seed():
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = '0'

    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()

#reset_random_seed()
# Adapted from:
# https://stackoverflow.com/questions/45230448/how-to-get-reproducible-result-when-running-keras-with-tensorflow-backend
# AND
# https://stackoverflow.com/questions/61078946/how-to-get-reproducible-results-keras-tensorflow

### Define Metrics

In [13]:
METRICS = [
      #tf.keras.metrics.TruePositives(name='tp'),
      #tf.keras.metrics.FalsePositives(name='fp'),
      #tf.keras.metrics.TrueNegatives(name='tn'),
      #tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      #tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      #tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

### Define string_lookup for One Hot Encoding the labels/type column

In [14]:
string_lookup = tf.keras.layers.StringLookup(output_mode='one_hot')
string_lookup.adapt(data["type"]) #TODO: ensure this doesn't use efficelty use test dataset for training

### Define Other Helper Functions

In [31]:
def custom_standardization(input_data):
    return tf.strings.lower(input_data)

def get_normalization_layer(feature_name):
    normalization_layer = tf.keras.layers.Normalization(axis=None)
    normalization_layer.adapt(data[feature_name].astype(np.float32)) #TODO: ensure this doesn't use efficelty use test dataset for training
    return normalization_layer

def get_vectorize_layer(feature_name):
    vectorize_layer = tf.keras.layers.TextVectorization(
        standardize=custom_standardization,
        max_tokens=max_features,
        output_mode='int',
        output_sequence_length=sequence_length
    )
    vectorize_layer.adapt(data[feature_name]) #TODO: ensure this doesn't use efficelty use test dataset for training
    return vectorize_layer

def get_string_lookup(feature_name):
    lookup = tf.keras.layers.StringLookup(
        output_mode='one_hot',
        max_tokens=sequence_length,
        pad_to_max_tokens=True,
    )
    lookup.adapt(data[feature_name]) #TODO: ensure this doesn't use efficelty use test dataset for training
    return lookup

In [38]:
tf.keras.layers.Flatten()(tf.keras.layers.Embedding(max_features, 5)(get_vectorize_layer("aux")(["test", "tes", "hi"])))

<tf.Tensor: shape=(3, 5000), dtype=float32, numpy=
array([[-0.01991978, -0.0091613 ,  0.02360929, ..., -0.034228  ,
         0.0440397 ,  0.03828735],
       [-0.01991978, -0.0091613 ,  0.02360929, ..., -0.034228  ,
         0.0440397 ,  0.03828735],
       [-0.01991978, -0.0091613 ,  0.02360929, ..., -0.034228  ,
         0.0440397 ,  0.03828735]], dtype=float32)>

### Define Get Model Function

In [29]:
def get_model(feature_names=feature_names):
    
    inputs = []
    outputs = []
    for i in range(len(feature_names)):
        
        feature_name = feature_names[i]
        flatten = tf.keras.layers.Flatten()
        if(feature_name == "latitude" or feature_name == "longitude"):
            #outputs.append(flatten(get_normalization_layer(feature_name)(tf.strings.to_number(inputs[:,i], out_type=tf.dtypes.float32))))
            inputs.append(tf.keras.Input(shape=(1,), dtype=tf.string, name=feature_name))
            outputs.append(flatten(get_normalization_layer(feature_name)(tf.strings.to_number(inputs[i], out_type=tf.dtypes.float32))))
            
            
        else:
            embedding_layer = tf.keras.layers.Embedding(max_features, 5)
            if(feature_name == "words" or feature_name == "aux"):
                layer = get_vectorize_layer(feature_name)
            else:
                layer = get_string_lookup(feature_name)
            #outputs.append(tf.keras.layers.Flatten()(embedding_layer(layer(flatten(inputs[:,i])))))
            
            inputs.append(tf.keras.Input(shape=(1,), dtype=tf.string, name=feature_name))
            outputs.append(flatten(embedding_layer(layer(tf.keras.layers.Flatten()(inputs[i])))))
    
    outputs = tf.concat(outputs, axis=-1)
    

    sequential_model = tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='relu'),
        #tf.keras.layers.Dropout(0.5),
        #tf.keras.layers.Dense(5, activation='relu'),
        tf.keras.layers.Dense(5)
    ])

    
    model = tf.keras.Model(inputs=inputs, outputs=sequential_model(outputs) )

    
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=METRICS
    )
    
    model.summary()
    sequential_model.summary()
    
    return model

## Train Model

### Define Dataset Getter Functions

In [24]:
def get_dataset(data=data, feature_names=feature_names):
    X = data[feature_names]
    y = (string_lookup(list(data["type"])))
    #print(str(X.to_dict())[0:1000])
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    
    train_and_validation, test = tf.keras.utils.split_dataset(dataset, right_size=test_size, shuffle=True, seed=seed)
    
    train, validation = tf.keras.utils.split_dataset(dataset, right_size=validation_size, shuffle=True, seed=seed)
    
    
    print(train)
    
    return train, validation, test
    #return dataset
    
    
def get_X_data(data=data, feature_names=feature_names):
    X = data[feature_names]
    return X

def get_y_data(data=data):
    y = string_lookup(list(data["type"]))
    return y

def get_split_datasets(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
    return X_train, X_test, y_train, y_test

### Define Train Model Function

In [26]:
def gen(X, y):
    X_out = {}
    y_out = tf.reshape(y, (1,5,))
    for i in range(len(feature_names)):
        feature_name = feature_names[i]
        #print(feature_name, tf.reshape(X[i], (1,)))
        X_out[feature_name] = tf.reshape(X[i], (1,))
    return X_out, y_out

def train_model(model, train, validation):
    #kfold = KFold(n_splits=folds, shuffle=True, random_state=seed)

    histories = []
    
    
    y_classes = np.argmax(np.concatenate([y for _, y in train.map(gen)]), axis=1)
    y_labels = np.unique(y_classes)
    class_weights = compute_class_weight(class_weight='balanced', classes=y_labels, y=y_classes)
    class_weights = dict(zip(y_labels, class_weights))
    class_weights[0] = 0
    
    print(train.map(gen))
    history = model.fit(
        train.map(gen),
        validation_data=validation.map(gen),
        epochs=epochs,
        #batch_size=batch_size,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)], #TODO: Try commenting this out
        class_weight=class_weights
    )
    histories.append(history)
        
    return histories

### Generate Feature Combination subsets

In [19]:
def generate_feature_combination_subsets():
    feature_names_subsets = []
    for N in range(len(feature_names) + 1):
         for feature_names_subset in itertools.combinations(feature_names, N): # adapted from: https://stackoverflow.com/questions/464864/get-all-possible-2n-combinations-of-a-list-s-elements-of-any-length
            feature_names_subset = list(feature_names_subset)
            if(len(feature_names_subset) > 0):
                feature_names_subsets.append(feature_names_subset)
    return feature_names_subsets

### Train and test all feature combinations

In [52]:
!mkdir -p saved_models
models = []
feature_names_subsets = generate_feature_combination_subsets()
for i in tqdm(range(len(feature_names_subsets)-1, 0, -1)):
    #reset_random_seed()
    feature_names_subset = feature_names_subsets[i]
    #print(feature_names_subset)
    #print("New Model: "+",".join(feature_names_subset))
    
    train, validation, test = get_dataset(data, feature_names_subset)
    
    print( str([ X for X, y in train.map(gen) ])[0:10000] )
    #model = get_model(feature_names_subset)
    #histories = train_model(model, train, validation)
    #model.evaluate(test.map(gen))
    #model.save("saved_model/"+",".join(feature_names_subset))
    break
    
    #X = get_X_data(data, feature_names_subset)
    # y = get_y_data(data)
    # X_train, X_test, y_train, y_test = get_split_datasets(X, y)
    # model = get_model(feature_names_subset)
    # histories = train_model(model, X_train, y_train)
    # model.evaluate(X_test, y_test, batch_size=batch_size)
    # tf.saved_model.save(model, "saved_model/"+",".join(feature_names_subset))
    # models.append({"features_names_subset":feature_names_subset, "model":model, "histories":histories})

  0%|                                                                                                                                                                                      | 0/254 [00:00<?, ?it/s]2023-04-25 13:42:22.079832: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [789,5]
	 [[{{node Placeholder/_1}}]]


<_PrefetchDataset element_spec=(TensorSpec(shape=(8,), dtype=tf.string, name=None), TensorSpec(shape=(5,), dtype=tf.float32, name=None))>


2023-04-25 13:42:22.666783: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [631,5]
	 [[{{node Placeholder/_1}}]]
  0%|                                                                                                                                                                                      | 0/254 [00:01<?, ?it/s]

[{'words': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b' Baseball Card Shopper Bob Dernier Baseball Card Prices Baseball card prices and checklists for Topps cards Bowman cards Fleer cards Donruss cards Leaf cards Baseball Cards By Year and Manufacturer Bob Dernier Available Bob Dernier Baseball Cards and Values View All Bob Dernier Cards Find the best deal on baseball cards you are searching for YearBrandCard PlayerTeamConditionPrice Value 1983 Topps 43 Bob Dernier Phillies nrmt mt 0 50 1984 Topps 358 Bob Dernier Phillies nrmt mt 0 50 1985 Topps 589 Bob Dernier Cubs nrmt mt 0 50 1986 Topps 188 Bob Dernier Cubs nrmt mt 0 50 1987 Topps 715 Bob Dernier Cubs nrmt mt 0 40 0 50 1989 Topps 418 Bob Dernier Phillies nrmt mt 0 50 1990 Topps 204 Bob Dernier Phillies nrmt mt 0 40 0 50 Home Terms of Use Privacy Policy Site Map Contact Us BaseballCardShopper com does not display complete baseball card prices and card listings for any vendor We update price listings regularly but vendor pri

## Evaluate Trained Model

### Define helper functions

In [ ]:
def one_hot_encoding_to_type_string(one_hot_encoding):
    return string_lookup.get_vocabulary()[np.argmax(tf.nn.softmax(one_hot_encoding))]

### Define visualization functions

In [ ]:
def plot_epochs_vs_accuracy_and_loss(model, histories):
    acc = []
    val_acc = []
    loss = []
    val_loss = []
    for history in histories:
        acc = acc + history.history['accuracy']
        val_acc = val_acc + history.history['val_accuracy']

        loss = loss + history.history['loss']
        val_loss = val_loss + history.history['val_loss']


    epochs_range = range(len(acc))

    pyplot.figure(figsize=(8, 8))
    pyplot.subplot(1, 2, 1)
    pyplot.plot(epochs_range, acc, label='Training Accuracy')
    pyplot.plot(epochs_range, val_acc, label='Validation Accuracy')
    pyplot.legend(loc='lower right')
    pyplot.title('Training and Validation Accuracy')

    pyplot.subplot(1, 2, 2)
    pyplot.plot(epochs_range, loss, label='Training Loss')
    pyplot.plot(epochs_range, val_loss, label='Validation Loss')
    pyplot.legend(loc='upper right')
    pyplot.title('Training and Validation Loss')
    pyplot.show()

In [ ]:
def show_prediction_vs_actual_dataframe_table(model):
    urls = [ data.loc[i, "url"] for i in range(len(X_test)) ]
    predictions = [ one_hot_encoding_to_type_string(model.predict(X_test)[i]) for i in range(len(X_test)) ]
    actuals = [ one_hot_encoding_to_type_string(y_test[i]) for i in range(len(y_test)) ]

    df = pd.DataFrame(np.array([urls, actuals, predictions]).T, columns=["url", "actual", "predicted"])
    df.reset_index(drop=True)
    return df[df["actual"] == df["predicted"]]

In [ ]:
for m in models:
    features_names_subset = m["features_names_subset"]
    model = m["model"]
    histories = m["histories"]
    print(features_names_subset)
    plot_epochs_vs_accuracy_and_loss(model, histories)
    show_prediction_vs_actual_dataframe_table(model)

## Predict More (Test) Data

In [ ]:
def predict(url):
    one = download_one(url)
    if(one is None or one["status_code"] != 200):
        return "Failed to get data for '" + url + "'"
    one = pd.DataFrame([one.values()], columns=list(one))
    one["redirects"] = one["redirects"].astype(str)
    one["latitude"] = one["latitude"].astype(str)
    one["longitude"] = one["longitude"].astype(str)
    
    outputs = []
    for m in models:
        features_names_subset = m["features_names_subset"]
        model = m["model"]
        histories = m["histories"]
        outputs.append(features_names_subset+": "+one_hot_encoding_to_type_string(model.predict(one[feature_names])))
    return "\n".join(outputs)

### Test URLs not in dataset

In [ ]:
print(predict("https://cnn.com/"))
print(predict("https://google.com/"))
print(predict("https://disneyplus.com/"))
print(predict("https://uvm.edu/"))
print(predict("https://en.wikipedia.org/"))

In [ ]:
print(predict("http://www.fwuits.monster/5576HU239G5U8qn613JT25e29A512cY17UFEGsvtwfDm-mEGsi8dRhomSdo6q_s1v06SJOHzt/caretaker-instrumentally"))
print(predict("http://www.luieis.beauty/attendee-sinfulness/d8c5V2z395ig86k13i21f62LK4ee2s17QFEGsvtwfDm-mEGsi11dR0om9Reoje6Vok1B08jUHtp1oL"))
print(predict("http://www.eiyeid.monster/e3f6K2Q3v95QF8u617c_OTR21f2Peo4ed9T17tFEGsvtwfDm-mEGsi11zRTom9QKoMe6V1Q0_m7aXXqHto/telephoner-plantation"))
print(predict("http://irever.live/r.php?q=NzkyNjM0NDs2MTg3OTsxMDA2OzM0OzI7MjAyMy0wMy0yMSAxNTozMDowNTs5OzE7bDs7"))
print(predict("https://click.yescaloriedietplan.com/?t=c&ids=NDM5NjExOTY3__NDIxMg%3D%3D__OTg0ODE1ODc%3D__OTAx__1102&url=aHR0cHMlM0ElMkYlMkZ0cmFja2luZy5oZWFsdGhpZXJsaWZ0LmNvbSUyRnpwdGk="))
print(predict("https://links.besttacticalknife.net/a/1625/click/4198/483611/747064e7195ac13a0f72ab11a22cb3f2ad8fa72c/4ec46075f0b8612b13e660aefd0511d442a49206"))
print(predict("http://www.folifort.email/l/lt1CK11667E122SG/5494NX7579O10146YB349N80481934SK3221472804"))

# TODO
1. Train and test a model on every combination of features to see what combinations are most important. - In Progress
2. Create a constant seed to reduce random noise-based accuracy changing between models - Done
3. Crate a confusion matrix

## Notes for Writeup
### Limitation
1. Redirects being treated as catagorical value